# Testing the Musical MDN Code

This script trains and evaluates the `musical_mdn` model on synthetic sample data. First, 50000 points of sample data are generated, then a small network is trained on this data for 30 epochs. Finally, the network is run in unconstrained generation mode to examine the results directly.

The `musical_mdn` includes a 2D mixture density recurrent neural network (MDRNN) model with a loss function defined by Tensorflow's included probability distribution functions. 

### Import modules

In [ ]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from context import *
import keras
import tensorflow as tf

## Training Test

- generate synthetic training data (noisy sine).
- initialise the ANN (`EmpiRNN`).
- Train for a few epochs.
- Plot the losses.

In [ ]:
x_t_log = empi_mdrnn.generate_data()
slices = empi_mdrnn.slice_sequence_examples(x_t_log, 121, step_size=10)
X, y = empi_mdrnn.seq_to_overlapping_format(slices)


net = empi_mdrnn.EmpiRNN(mode = empi_mdrnn.NET_MODE_TRAIN, n_hidden_units = 128, n_mixtures = 10, batch_size = 100, sequence_length = 120)
history = net.train(X, y, num_epochs=10, saving=False)
## Plot the losses.

In [ ]:
## Plot the losses.
loss_df = pd.DataFrame(history.history)
loss_df.plot()
print("Losses for each epoch.")

In [ ]:
# Transfer model to decoder.
weights = net.model.get_weights()

In [ ]:
net = empi_mdrnn.EmpiRNN(mode = empi_mdrnn.NET_MODE_RUN, n_hidden_units = 128, n_mixtures = 10, layers=2)
net.model.set_weights(weights)

## Evaluation Test

Predict 10000 Datapoints and plot some segments.

In [ ]:
# compute_graph = tf.Graph()
# with compute_graph.as_default():
#     sess = tf.Session()

In [ ]:
## Evaluation Test:
## Predict 10000 Datapoints.
net = empi_mdrnn.EmpiRNN(mode = empi_mdrnn.NET_MODE_RUN, n_hidden_units = 128, n_mixtures = 5, layers=2)

In [ ]:
# Load human trained model.
net.load_model(model_file = '../models/empi_mdrnn-layers2-units128-mixtures5-scale10-E84-VL-3.68.hdf5')

In [ ]:
first_touch = empi_mdrnn.random_sample()
print("Test Input:",first_touch)
#with tf.Session() as sess
perf = net.generate_performance(first_touch,10000) #,sess)
print("Test Output:")
perf_df = pd.DataFrame({'t':perf.T[0], 'x':perf.T[1]})
perf_df['time'] = perf_df.t.cumsum()
#plt.show(perf_df.plot('time','x',kind='scatter'))
plt.plot(perf_df.time, perf_df.x, '.r-')
plt.show()
print(perf_df.describe())
## Investigate Output
window = 100
for n in [1000,2000,3000,4000,5000,6000]:
    print("Window:", str(n),'to',str(n+window))
    plt.plot(perf_df[n:n+window].time, perf_df[n:n+window].x, '.r-')
    plt.show()